# Geen lokale bestanden

In sommige implementaties is het gebruik van een locale `.yaml` bestand niet wenselijk. De configuratie kan ook geheel in de code worden gezet. Dit kan handig zijn voor kleinere projecten of wanneer je geen aparte configuratiebestanden wilt beheren.

In [ ]:
from pathlib import Path

from toolbox_continu_inzicht import Config

## 1. Voorbeeld met CSV - data adapter (classiek)
Lees de configuratie in:

Dit leest een configuratie bestand `test_config.yaml` in. 
Hier definieren we een aantal data adapaters die lezen/schrijven mogelijk maken:

```yaml
GlobalVariables:
    rootdir: 'tests/src/base/data_sets'

DataAdapter:
    postgresql_database:
        database: 'citoolbox'
        schema: 'citoolbox_schema'
        
    my_csv_in: 
        type: csv
        file: 'test_csv_in.csv'
    my_csv_in_delim: 
        type: csv
        file: 'test_csv_in_delim.csv'
        sep: ';'
    MyCSV_out: 
        type: csv
        file: 'test_csv_out.csv'
    MyPostgresql: 
        type: postgresql_database
        table: data_test
    MyNetCDF_in:
        type: netcdf
        file: 'test_netcdf_in.nc'
    MyNetCDF_out:
        type: netcdf
        file: 'test_netcdf_out.nc'

...

```

In [ ]:
test_data_sets_path = Path.cwd() / "data_sets"
config_via_yaml = Config(config_path=test_data_sets_path / "test_config.yaml")
config_via_yaml.lees_config()

vervolgens geven we de configuratie mee aan de data adapter

In [ ]:
from toolbox_continu_inzicht import DataAdapter
from toolbox_continu_inzicht.proof_of_concept import example_module

data_adapter_via_yaml = DataAdapter(config=config_via_yaml)


keer_twee_yaml = example_module.ValuesTimesTwo(data_adapter=data_adapter_via_yaml)
keer_twee_yaml.run(input="mycsv_in", output="mycsv_out")

input_df = keer_twee_yaml.df_in  # Bewaren we voor straks

De configuratie die we hadden ingelezen kunnen we ook bekijken en vervolgens gebruiken om de data adapter te initialiseren vanuit python.

In [ ]:
config_via_yaml.global_variables

In [ ]:
config_via_yaml.data_adapters

## 2. Voorbeeld geheel in python - wel data adapters
Maak de configuratie geheel in python als dictionary, maar wel nog veel structuur

In [ ]:
import datetime


python_global_variables = {
    "rootdir": "data_sets",
    "calc_time": datetime.datetime(2025, 8, 12, 11, 0, tzinfo=datetime.timezone.utc),
}

In [ ]:
python_data_adapters = {
    "default_options": {
        "postgresql_database": {"database": "citoolbox", "schema": "citoolbox_schema"},
        "csv": {"sep": ","},
    },
    "mycsv_in": {"type": "csv", "file": "test_csv_in.csv", "sep": ","},
    "mycsv_in_delim": {"type": "csv", "file": "test_csv_in_delim.csv", "sep": ";"},
    "mycsv_out": {"type": "csv", "file": "test_csv_out.csv", "sep": ","},
    "mypostgresql": {
        "type": "postgresql_database",
        "table": "data",
        "database": "citoolbox",
        "schema": "citoolbox_schema",
    },
    "mynetcdf_in": {"type": "netcdf", "file": "test_netcdf_in.nc"},
    "mynetcdf_out": {"type": "netcdf", "file": "test_netcdf_out.nc"},
    "my_df_python": {"type": "python"},
}

In [ ]:
current_path = Path.cwd()
config_via_dict = Config(config_path=current_path)
config_via_dict.global_variables = python_global_variables
config_via_dict.data_adapters = python_data_adapters

In [ ]:
data_adapter_via_python = DataAdapter(config=config_via_dict)

In [ ]:
data_adapter_via_python.set_dataframe_adapter("my_df_python", input_df)

In [ ]:
keer_twee_python = example_module.ValuesTimesTwo(data_adapter=data_adapter_via_python)
keer_twee_python.run(input="my_df_python", output="my_df_python")

In [ ]:
df_in_functie2 = keer_twee_python.df_out
df_in_functie2

## 3. Voorbeeld geheel in python  - zonder data adapters
Maak de configuratie geheel in python als dictionary, maar weinig structuur.

In [ ]:
python_global_variables2 = {}
python_data_adapters2 = {}
current_path = Path.cwd()
config_via_dict2 = Config(config_path=current_path)
config_via_dict2.global_variables = python_global_variables2
config_via_dict2.data_adapters = python_data_adapters2
data_adapter_via_python2 = DataAdapter(config=config_via_dict2)

In [ ]:
data_adapter_via_python2.set_dataframe_adapter(
    "my_df_python_2", df_in_functie2, if_not_exist="create"
)
data_adapter_via_python2.set_dataframe_adapter(
    "my_df_python_out", df_in_functie2, if_not_exist="create"
)

In [ ]:
keer_twee_python2 = example_module.ValuesTimesTwo(data_adapter=data_adapter_via_python2)

In [ ]:
keer_twee_python2.run(input="my_df_python_2", output="my_df_python_out")
keer_twee_python2.df_out